### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [35]:
total_number_schools = len(school_data_complete["school_name"].unique())
total_number_students = len(school_data_complete["student_name"].unique())
total_budget = school_data_complete["budget"].unique().sum()
avg_math_score = school_data_complete["math_score"].mean()
avg_reading_score = school_data_complete["reading_score"].mean()
overall_passing_rate = avg_math_score+avg_reading_score/2
cond = (school_data_complete["math_score"] >= 70)
students_70_math = school_data_complete[cond]
math_70_count = len(students_70_math["student_name"].unique())
percentage_students_passing_mathscore = math_70_count/total_number_students *100
cond1 = (school_data_complete["reading_score"] >= 70)
students_70_reading = school_data_complete[cond1]
reading_70_count = len(students_70_reading["student_name"].unique())
percentage_students_passing_readingscore = reading_70_count/total_number_students *100
cols = {
    "Total Schools" : total_number_schools,
    "Total Students" : total_number_students,
    "Total Budget" : total_budget,
    "Average Math Score" : avg_math_score,
    "Average Reading Score" : avg_reading_score,
    "% Passing Math" : percentage_students_passing_mathscore,
    "% Passing Reading" : percentage_students_passing_readingscore,
    "% Overall Passing Rate" : overall_passing_rate
}
summary = pd.DataFrame(data = cols, index = [1])
summary.head()
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [232]:
# Calculating % Passing Math and % Passing Reading
tot_math_studs = school_data_complete.groupby("school_name").agg({"math_score" : "count"})
tot_reading_studs = school_data_complete.groupby("school_name").agg({"reading_score" : "count"})
cond1 = (school_data_complete["math_score"] >= 70)
cond2 = (school_data_complete["reading_score"] >= 70)
df1 = school_data_complete[cond1]
df1 = df1.groupby("school_name").agg({"math_score" : "count"})
math_table = pd.merge(tot_math_studs, df1, on="school_name")
math_table["% Passing Math"] = math_table["math_score_y"]/math_table["math_score_x"]
math_table = round(math_table["% Passing Math"].multiply(100),2)
math_table = pd.DataFrame(math_table)
math_table = math_table.reset_index()

df2 = school_data_complete[cond2]
df2 = df2.groupby("school_name").agg({"reading_score" : "count"})
reading_table = pd.merge(tot_reading_studs, df2, on="school_name")
reading_table["% Passing Reading"] = reading_table["reading_score_y"]/reading_table["reading_score_x"]
reading_table = round(reading_table["% Passing Reading"].multiply(100),2)
reading_table = pd.DataFrame(reading_table)
reading_table = reading_table.reset_index()

# Calculating based on grouping of school name - total Students, budget etc

li =[ "school_name","budget","type"]
group = school_data_complete.groupby(li, as_index = False).agg({'student_name': "count", 'math_score' :'mean', 'reading_score' : 'mean'})
per_student_budget = group["budget"]/group["student_name"]
group["Student Budget"] = per_student_budget
group = pd.merge(math_table,group, how ="outer", on = "school_name")
group = pd.merge(reading_table,group, how ="outer", on = "school_name")
group["Overall Passing"] = round ((group["% Passing Reading"] + group["% Passing Math"])/2,2)
group.head()
colum = {
    "school_name" : "School Name",
    "type" : "School Type",
    "budget" : "Total School Budget",
    "Student Budget" : "Per Student Budget",
    "math_score" : "Average Math Score",
    "reading_score" : "Average Reading Score",
    "% Passing Math" : "% Passing Math",
    "% Passing Reading" : "% Passing Reading",
    "% Overall Passing Rate" : "% Overall Passing Rate",
    "student_name" : "Total Students"
}
group = group.rename(columns = colum)
group.head()

,School Name,% Passing Reading,% Passing Math,Total School Budget,School Type,Total Students,Average Math Score,Average Reading Score,Per Student Budget,Overall Passing
0,Bailey High School,81.93,66.68,3124928,District,4976,77.048432,81.033963,628.0,74.31
1,Cabrera High School,97.04,94.13,1081356,Charter,1858,83.061895,83.975780,582.0,95.58
2,Figueroa High School,80.74,65.99,1884411,District,2949,76.711767,81.158020,639.0,73.36
3,Ford High School,79.30,68.31,1763916,District,2739,77.102592,80.746258,644.0,73.81
4,Griffin High School,97.14,93.39,917500,Charter,1468,83.351499,83.816757,625.0,95.26


## Top Performing Schools (By Passing Rate)
* Sort and display the top five schools in overall passing rate

In [204]:
group.sort_values(by = "Overall Passing", ascending = False)

,School Name,% Passing Reading,% Passing Math,Total School Budget,School Type,Total Students,Average Math Score,Average Reading Score,Per Student Budget,Overall Passing
1,Cabrera High School,97.04,94.13,1081356,Charter,1858,83.061895,83.975780,582.0,95.58
12,Thomas High School,97.31,93.27,1043130,Charter,1635,83.418349,83.848930,638.0,95.29
9,Pena High School,95.95,94.59,585858,Charter,962,83.839917,84.044699,609.0,95.27
4,Griffin High School,97.14,93.39,917500,Charter,1468,83.351499,83.816757,625.0,95.26
13,Wilson High School,96.54,93.87,1319574,Charter,2283,83.274201,83.989488,578.0,95.21
14,Wright High School,96.61,93.33,1049400,Charter,1800,83.682222,83.955000,583.0,94.97
11,Shelton High School,95.85,93.87,1056600,Charter,1761,83.359455,83.725724,600.0,94.86
6,Holden High School,96.25,92.51,248087,Charter,427,83.803279,83.814988,581.0,94.38
0,Bailey High School,81.93,66.68,3124928,District,4976,77.048432,81.033963,628.0,74.31
3,Ford High School,79.30,68.31,1763916,District,2739,77.102592,80.746258,644.0,73.81


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [205]:
group.sort_values(by = "Overall Passing", ascending = True)

,School Name,% Passing Reading,% Passing Math,Total School Budget,School Type,Total Students,Average Math Score,Average Reading Score,Per Student Budget,Overall Passing
10,Rodriguez High School,80.22,66.37,2547363,District,3999,76.842711,80.744686,637.0,73.30
2,Figueroa High School,80.74,65.99,1884411,District,2949,76.711767,81.158020,639.0,73.36
7,Huang High School,81.32,65.68,1910635,District,2917,76.629414,81.182722,655.0,73.50
8,Johnson High School,81.22,66.06,3094650,District,4761,77.072464,80.966394,650.0,73.64
3,Ford High School,79.30,68.31,1763916,District,2739,77.102592,80.746258,644.0,73.81
5,Hernandez High School,80.86,66.75,3022020,District,4635,77.289752,80.934412,652.0,73.81
0,Bailey High School,81.93,66.68,3124928,District,4976,77.048432,81.033963,628.0,74.31
6,Holden High School,96.25,92.51,248087,Charter,427,83.803279,83.814988,581.0,94.38
11,Shelton High School,95.85,93.87,1056600,Charter,1761,83.359455,83.725724,600.0,94.86
14,Wright High School,96.61,93.33,1049400,Charter,1800,83.682222,83.955000,583.0,94.97


## Math Scores by Grade

In [196]:
school_data_complete.head()
cond9 = (school_data_complete["grade"] == "9th")
gr9 = school_data_complete[cond9]
gr9 = gr9.groupby("school_name").agg({"math_score" : "mean"})
cond10 = (school_data_complete["grade"] == "10th")
gr10 = school_data_complete[cond10]
gr10 = gr10.groupby("school_name").agg({"math_score" : "mean"})
cond11 = (school_data_complete["grade"] == "11th")
gr11 = school_data_complete[cond11]
gr11 = gr11.groupby("school_name").agg({"math_score" : "mean"})
cond12 = (school_data_complete["grade"] == "12th")
gr12 = school_data_complete[cond12]
gr12 = gr12.groupby("school_name").agg({"math_score" : "mean"})
mgr = pd.merge(gr9,gr10, on = "school_name", suffixes = (9, 10))
mgr = pd.merge(mgr,gr11, on = "school_name")
mgr = pd.merge(mgr,gr12, on = "school_name")
col = {
    "math_score9" : "9th",
    "math_score10" : "10th",
    "math_score_x" : "11th",
    "math_score_y" : "12th"
}
mgr = mgr.rename(columns = col)
mgr.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


## Reading Score by Grade 
* Perform the same operations as above for reading scores

In [197]:
school_data_complete.head()
cond9 = (school_data_complete["grade"] == "9th")
gr9 = school_data_complete[cond9]
gr9 = gr9.groupby("school_name").agg({"reading_score" : "mean"})
cond10 = (school_data_complete["grade"] == "10th")
gr10 = school_data_complete[cond10]
gr10 = gr10.groupby("school_name").agg({"reading_score" : "mean"})
cond11 = (school_data_complete["grade"] == "11th")
gr11 = school_data_complete[cond11]
gr11 = gr11.groupby("school_name").agg({"reading_score" : "mean"})
cond12 = (school_data_complete["grade"] == "12th")
gr12 = school_data_complete[cond12]
gr12 = gr12.groupby("school_name").agg({"reading_score" : "mean"})
mgr = pd.merge(gr9,gr10, on = "school_name", suffixes = (9, 10))
mgr = pd.merge(mgr,gr11, on = "school_name")
mgr = pd.merge(mgr,gr12, on = "school_name")
col = {
    "reading_score9" : "9th",
    "reading_score10" : "10th",
    "reading_score_x" : "11th",
    "reading_score_y" : "12th"
}
mgr = mgr.rename(columns = col)
mgr.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [230]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [238]:
group["Per Student Budget"]
pd.cut(group["Per Student Budget"], bins = spending_bins, labels=group_names, include_lowest=True)
group["Spend Category"] = pd.cut(group["Per Student Budget"], bins = spending_bins, labels=group_names, include_lowest=True)
sp_grouped = group.groupby("Spend Category")
sp_grouped.max()

,School Name,% Passing Reading,% Passing Math,Total School Budget,School Type,Total Students,Average Math Score,Average Reading Score,Per Student Budget,Overall Passing
Spend Category,,,,,,,,,,
<$585,Wright High School,97.04,94.13,1319574,Charter,2283,83.803279,83.989488,583.0,95.58
$585-615,Shelton High School,95.95,94.59,1056600,Charter,1761,83.839917,84.044699,609.0,95.27
$615-645,Thomas High School,97.31,93.39,3124928,District,4976,83.418349,83.848930,644.0,95.29
$645-675,Johnson High School,81.32,66.75,3094650,District,4761,77.289752,81.182722,655.0,73.81


## Scores by School Size

* Perform the same operations as above, based on school size.

In [241]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [242]:
group["size category"] = pd.cut(group["Total Students"], bins = size_bins, labels = group_names, include_lowest = True)
szgroup = group.groupby("size category").max()
szgroup

,School Name,% Passing Reading,% Passing Math,Total School Budget,School Type,Total Students,Average Math Score,Average Reading Score,Per Student Budget,Overall Passing,Spend Category
size category,,,,,,,,,,,
Small (<1000),Pena High School,96.25,94.59,585858,Charter,962,83.839917,84.044699,609.0,95.27,$585-615
Medium (1000-2000),Wright High School,97.31,94.13,1081356,Charter,1858,83.682222,83.975780,638.0,95.58,$615-645
Large (2000-5000),Wilson High School,96.54,93.87,3124928,District,4976,83.274201,83.989488,655.0,95.21,$645-675


## Scores by School Type

* Perform the same operations as above, based on school type.

In [246]:
group.head()
tygroup = group.groupby("School Type").max()
tygroup

,School Name,% Passing Reading,% Passing Math,Total School Budget,Total Students,Average Math Score,Average Reading Score,Per Student Budget,Overall Passing,Spend Category,size category
School Type,,,,,,,,,,,
Charter,Wright High School,97.31,94.59,1319574,2283,83.839917,84.044699,638.0,95.58,$615-645,Large (2000-5000)
District,Rodriguez High School,81.93,68.31,3124928,4976,77.289752,81.182722,655.0,74.31,$645-675,Large (2000-5000)
